In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# ================= IMPORTS =================
!pip install segmentation_models_pytorch
import os
import glob
import torch
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from torchvision import transforms
import segmentation_models_pytorch as smp
from tqdm import tqdm
import torch.nn.functional as F
import gdown

#data path please change accordingly
#Run the notebook on kaggle environment for no requirements problem
TEST_IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded"

ORIG_H, ORIG_W = 540, 960
MODEL_H, MODEL_W = 544, 960

THRESHOLD = 0.48
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MASK_OUTDIR = "binary_masks"
CSV_OUT = "submission.csv"
os.makedirs(MASK_OUTDIR, exist_ok=True)
os.makedirs("weights", exist_ok=True)

UNET_URL = "https://drive.google.com/uc?id=1LfPfzxr828oQTsW5TR8wAkGt__N5H7Cn"
MIT_URL  = "https://drive.google.com/uc?id=1sfPWzTY_l_WIcU1FAg7E-MK6MkZn2cNJ"

UNET_PATH = "weights/unet.pth"
MIT_PATH  = "weights/mit.pth"

gdown.download(UNET_URL, UNET_PATH, quiet=False)
gdown.download(MIT_URL,  MIT_PATH,  quiet=False)

# ================= TRANSFORMS =================
img_tf_pad = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

img_tf_resize = transforms.Compose([
    transforms.Resize((MODEL_H, MODEL_W)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


def freeze(model):
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    return model

model_unet = freeze(
    smp.Unet("resnet34", encoder_weights=None, classes=1).to(DEVICE)
)
model_unet.load_state_dict(
    torch.load(UNET_PATH, map_location=DEVICE,  weights_only=False)["model_state_dict"]
)

model_mit = freeze(
    smp.Unet("mit_b3", encoder_weights="imagenet", classes=1).to(DEVICE)
)
model_mit.load_state_dict(
    torch.load(MIT_PATH, map_location=DEVICE,  weights_only=False)["model_state_dict"]
)


def rle_encode(mask):
    pixels = mask.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))


paths = sorted(glob.glob(os.path.join(TEST_IMG_DIR, "*.png")))
print(f"Found {len(paths)} test images")

rows = []

with torch.no_grad():
    for p in tqdm(paths):

        img = Image.open(p).convert("RGB")
        img_pad = transforms.functional.pad(img, (0,2,0,2))

        x_unet   = img_tf_pad(img_pad).unsqueeze(0).to(DEVICE)
        x_resize = img_tf_resize(img).unsqueeze(0).to(DEVICE)

        p_unet = torch.sigmoid(model_unet(x_unet))[0,0]
        p_mit  = torch.sigmoid(model_mit(x_resize))[0,0]

        p_unet = F.interpolate(
            p_unet[None,None], (ORIG_H, ORIG_W)
        )[0,0].cpu().numpy()

        p_mit = F.interpolate(
            p_mit[None,None], (ORIG_H, ORIG_W)
        )[0,0].cpu().numpy()

       
        ensemble = 0.3 * p_unet + 0.7 * p_mit
        final_mask = (ensemble > THRESHOLD).astype(np.uint8)

        rows.append({
            "image_id": os.path.splitext(os.path.basename(p))[0],
            "encoded_pixels": rle_encode(final_mask)
        })

        cv2.imwrite(
            os.path.join(MASK_OUTDIR, os.path.basename(p)),
            final_mask * 255
        )


pd.DataFrame(rows).to_csv(CSV_OUT, index=False)
print("✅ Inference complete")
print("Masks saved to:", MASK_OUTDIR)
print("Submission saved as:", CSV_OUT)


Downloading...
From (original): https://drive.google.com/uc?id=1LfPfzxr828oQTsW5TR8wAkGt__N5H7Cn
From (redirected): https://drive.google.com/uc?id=1LfPfzxr828oQTsW5TR8wAkGt__N5H7Cn&confirm=t&uuid=29e1b6eb-7179-4cbe-9d21-ff6d4ad4d3ce
To: /kaggle/working/weights/unet.pth
100%|██████████| 294M/294M [00:04<00:00, 61.7MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1sfPWzTY_l_WIcU1FAg7E-MK6MkZn2cNJ
From (redirected): https://drive.google.com/uc?id=1sfPWzTY_l_WIcU1FAg7E-MK6MkZn2cNJ&confirm=t&uuid=7593f2f1-2a13-44e3-95b9-a9741b105476
To: /kaggle/working/weights/mit.pth
100%|██████████| 569M/569M [00:06<00:00, 90.5MB/s] 


config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

Found 1 test images


100%|██████████| 1/1 [00:04<00:00,  4.83s/it]

✅ Inference complete
Masks saved to: binary_masks
Submission saved as: submission.csv
